# :chart_with_upwards_trend: Telco Churn Model - Part 1 (Data Loading and Exploration)

### Please run this first Notebook fully before running the second Notebook.

### First, add the `imbalanced-learn`, `snowflake-ml-python`, `altair`, `pandas`, and `numpy` packages from the package picker on the top right. We will be using these packages later in the notebook.

In this solution, we will play the role of a data scientist at a telecom company that wants to identify users who are at high risk of churning. To accomplish this, we need to build a model that can learn how to identify such users. We will demonstrate how to use Snowflake Notebook in conjunction with Snowflake/Snowpark to build a Random Forest Classifier to help us with this task.

### Prerequisites
- Familiarity with basic Python and SQL
- Familiarity with training ML models
- Familiarity with data science notebooks
- Go to the Snowflake sign-up page and register for a free account. After registration, you will receive an email containing a link that will take you to Snowflake, where you can sign in.

### What You'll Learn
- How to import/load data with Snowflake Notebook
- How to train a Random Forest with Snowpark ML model
- How to visualize the predicted results from the forecasting model
- How to build an interactive web app and make predictions on new users

## Importing Data
To pull our churn dataset into SnowSight notebooks, we will pull some parquet data from AWS S3.

In [ ]:
CREATE OR REPLACE STAGE TELCO_CHURN_EXTERNAL_STAGE_DEMO
    URL = 's3://sfquickstarts/notebook_demos/churn/';

CREATE FILE FORMAT IF NOT EXISTS MY_PARQUET_FORMAT TYPE = PARQUET COMPRESSION = SNAPPY;

CREATE TABLE if not exists TELCO_CHURN_RAW_DATA_DEMO USING TEMPLATE ( 
    SELECT ARRAY_AGG(OBJECT_CONSTRUCT(*)) 
    FROM 
        TABLE( INFER_SCHEMA( 
        LOCATION => '@TELCO_CHURN_EXTERNAL_STAGE_DEMO', 
        FILE_FORMAT => 'MY_PARQUET_FORMAT',
        FILES => 'telco_churn.parquet'
        ) 
    ) 
);

COPY INTO TELCO_CHURN_RAW_DATA_DEMO
FROM @TELCO_CHURN_EXTERNAL_STAGE_DEMO
FILES = ('telco_churn.parquet')
FILE_FORMAT = (
    TYPE=PARQUET,
    REPLACE_INVALID_CHARACTERS=TRUE,
    BINARY_AS_TEXT=FALSE
)
MATCH_BY_COLUMN_NAME=CASE_INSENSITIVE
ON_ERROR=ABORT_STATEMENT;

SELECT * FROM TELCO_CHURN_RAW_DATA_DEMO;

# Working with Data

We can start working with the data using our familiar data science libraries in Python.

In [ ]:
import pandas as pd
import numpy as np
import streamlit as st
import altair as alt
from imblearn.over_sampling import SMOTE 

import warnings
warnings.filterwarnings("ignore")

# Getting session
from snowflake.snowpark.context import get_active_session
session = get_active_session()
session.query_tag = {"origin":"sf_sit-is", 
                     "name":"churn_prediction", 
                     "version":{"major":1, "minor":0},
                     "attributes":{"is_quickstart":1, "source":"notebook"}}

telco_churn_snow_df = cell2.to_df()
telco_churn_snow_df

## Exploratory Data Analysis (EDA)

Machine learning models thrive on clean and well-organized data. To ensure our models perform at their best, we'll investigate our dataset to address any missing values and visualize the distributions of each column.

### Basic Summary Statistics

In [ ]:
telco_churn_snow_df.describe()

### Checking nulls with Pandas

In [ ]:
telco_churn_pdf = telco_churn_snow_df.to_pandas()
session.create_dataframe(telco_churn_pdf).write.save_as_table("telco_churn_pdf", mode="overwrite")
telco_churn_pdf.isnull().sum()

As can be seen, there is no null value in any of the feature columns.

### Visualizing Feature Distributions

In [ ]:
columns = telco_churn_pdf.columns
num_columns_for_display = 3
col1, col2 , col3 = st.columns(num_columns_for_display)
index = 0
for col in columns:
    source = pd.DataFrame(telco_churn_pdf[col])
    chrt = alt.Chart(source).mark_bar().encode(
    alt.X(f"{col}:Q", bin=True),
    y='count()',
    )
    if index % num_columns_for_display == 0:
        with col1: 
            st.altair_chart(chrt)
    elif index % num_columns_for_display == 1:
        with col2: 
            st.altair_chart(chrt)
    elif index % num_columns_for_display == 2:
        with col3: 
            st.altair_chart(chrt)
    index = index + 1

### Understanding Churn Rate - Imbalanced dataset

In [ ]:
telco_churn_snow_df.group_by('"Churn"').count()

If you want to understand a model, you need to know its weaknesses. When the target variable has one class that is much more frequent than the other, your data is imbalanced. This causes issues when evaluating models since both classes don't get equal attention.

In contrast to modeling an imbalanced dataset, a model trained on balanced data sees an equal amount of observations per class. By eliminating the imbalance, we also eliminate the model's potential to achieve high metric scores due to bias towards a majority class. This means that when we evaluate our model, the metrics can capture a better representation of how well the model does at making valuable predictions.

#### Comparing Big data processing with pandas v.s. Snowpark Dataframes

For the groupby aggregation query above, we used Snowpark dataframes to perform the operation. Snowpark's Dataframe API allows you to query and process data at scale in Snowflake. With Snowpark, you no longer have to convert your dataframes to pandas in memory. Snowpark lets process data in Snowflake without moving data to the system where your application code runs, and process at scale as part of the elastic and serverless Snowflake engine.

Below we look at how the query performance of the groupby aggregation with Snowpark v.s. pandas.

In [ ]:
import time
start = time.time()
telco_churn_snow_df.group_by('"Churn"').count()
end = time.time()
st.markdown(f"Total Time with Snowpark: {end-start}")

start = time.time()
telco_churn_snow_pdf = telco_churn_snow_df.to_pandas()
end_mid = time.time()
telco_churn_snow_pdf.groupby("Churn").count()
end = time.time()
st.markdown(f"Total Time with Pandas: {end-start}")

We can see that Snowpark runs much faster. This is because of the I/O overhead for converting a Snowpark dataframe to pandas. We can see that the bulk of the time spent is on I/O.

In [ ]:
st.markdown(f"I/O time to convert to Pandas dataframe: {end_mid-start}")
st.markdown(f"Processing time with Pandas dataframe: {end-end_mid}")
st.markdown(f"I/O account for {(end_mid-start)/(end-start)*100:.2f}% of processing time")

### This concludes the first Notebook. Please go to the second Notebook to continue this solution.